In [17]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB  
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import GradientBoostingClassifier
import sys
path="data/"
inputname="human.fasta"
outputname="human_jackknife_PSTNP"

In [18]:
'''
cross validation svm
'''
classifier="SVM"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV

y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0)
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=3
gap=0
columns_num=39
divided_num=1130
division_num=2260
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=divided_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value1=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value1[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value1=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value1[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    for jack_index,jack_data in enumerate(y_final_seq_value1):
        X_train1 = np.array(final_seq_value1)
        print X_train1.shape
        Y_train = list(map(lambda x: 1, xrange(len(X_train1) / 2)))
        Y2_train = list(map(lambda x: 0, xrange(len(X_train1) / 2)))
        Y_train.extend(Y2_train)
        Y_train = np.array(Y_train)

        if jack_index==0:
            Y_test=[0]
            X_test1 = np.array(y_final_seq_value1[1])
            X_train1=np.concatenate([X_train1,np.array(y_final_seq_value1[0]).reshape(1,columns_num)],axis=0)
            Y_train=np.concatenate([Y_train.reshape(division_num-2,1),np.array([1]).reshape(1,1)],axis=0)
        else:
            Y_test=[1]
            X_test1 = np.array(y_final_seq_value1[0])
            X_train1=np.concatenate([X_train1,np.array(y_final_seq_value1[1]).reshape(1,columns_num)],axis=0)
            Y_train=np.concatenate([Y_train.reshape(division_num-2,1),np.array(0).reshape(1,1)],axis=0)
        svc = svm.SVC(probability=True)
        parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
        clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
        Y_train=np.array(list(Y_train))
        Y_train1=list([1 if x==1 else 0 for x in Y_train ])
        clf.fit(X_train1, Y_train1)
        C=clf.best_params_['C']
        y_pred_prob=clf.predict_proba(X_test1)

        gamma=clf.best_params_['gamma']
        print 'c:',C,'gamma:',gamma
        y_pred=clf.predict(X_test1)

        y_pred_prob_all.extend(y_pred_prob)
        y_pred_all.extend(y_pred)
        Y_all.append(Y_test)
        
all_y=[np.array(Y_all).astype(int).T[0],np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_jackknife_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T[0], list(np.array(y_pred_prob_all).astype(list)[:,1]))

roc_auc = metrics.roc_auc_score(np.array(Y_all).T[0],np.array(y_pred_prob_all).astype(list)[:,1])
ACC=metrics.accuracy_score(np.array(Y_all).T[0],list(np.array(y_pred_all).astype(int)))
print "ACC",ACC
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int))) 
F1_Score=metrics.f1_score(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int)))
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int)))
pos=TP+FN
neg=FP+TN


savedata=[str(X_train1.shape[1]),u"正："+str(pos)+u'负：'+str(neg),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN,
            SP, GM,F_measure,F1_Score,MCC,roc_auc,TP,
            FN,FP,TN]
final_out_to_excel.append(savedata)
print savedata
pd.DataFrame(final_out_to_excel).to_excel(path+'jackknife_'+classifier+"_"+outputname+'.xlsx',sheet_name="_crossvalidation",header=None,index=None)

(2258, 39)
c: 1.25992104989 gamma: 4.0
(2258, 39)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

c: 1.25992104989 gamma: 4.0
(2258, 39)
c: 1.25992104989 gamma: 4.0
(2258, 39)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

c: 1.25992104989 gamma: 4.0
(2258, 39)
c: 1.25992104989 gamma: 4.0
(2258, 39)


...

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

c: 1.25992104989 gamma: 4.0
(2258, 39)
c: 1.25992104989 gamma: 4.0
(2258, 39)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

c: 1.25992104989 gamma: 4.0
(2258, 39)


KeyboardInterrupt: 

In [16]:
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0)
roc_auc = metrics.roc_auc_score(np.array(Y_all).T[0],np.array(y_pred_prob_all).astype(list)[:,1])
ACC=metrics.accuracy_score(np.array(Y_all).T[0],list(np.array(y_pred_all).astype(int)))
print "ACC",ACC
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int))) 
F1_Score=metrics.f1_score(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int)))
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int)))
pos=TP+FN
neg=FP+TN


savedata=[str(X_train1.shape[1]),u"正："+str(pos)+u'负：'+str(neg),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN,
            SP, GM,F_measure,F1_Score,MCC,roc_auc,TP, FN,FP,TN]
final_out_to_excel.append(savedata)
print final_out_to_excel
pd.DataFrame(final_out_to_excel).to_excel(path+'jackknife_'+classifier+"_"+outputname+'.xlsx',sheet_name="_crossvalidation",header=None,index=None)

ACC 0.908849557522
[[u'\u7279\u5f81\u96c6', u'\u6837\u672c\u4e2a\u6570', u'\u5206\u7c7b\u5668', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP', u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC\u66f2\u7ebf\u9762\u79ef', u'tp', u'fn', u'fp', u'tn'], ['40', u'\u6b63\uff1a1130.0\u8d1f\uff1a1130.0', 'svmC:2.82842712475gamma:4.0', 0.90884955752212393, 1.0, 0.8176991150442477, 0.8176991150442477, 1.0, 0.9042671701683346, 0.89970788704965921, 0.89970788704965921, 0.83163499593695911, 0.92037042838123595, 924.0, 206.0, 0.0, 1130.0]]


In [ ]:
'''
cross validation GBDT
'''
classifier="GBDT"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    

    

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    
    GBDT = GradientBoostingClassifier(learning_rate=0.1,min_samples_split=300,min_samples_leaf=20,max_depth=8,max_features='sqrt')
    parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
    clf = GridSearchCV(GBDT, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    n_estimators=clf.best_params_['n_estimators']
    max_depth=clf.best_params_['max_depth']
    min_samples_split=clf.best_params_['min_samples_split']
    min_samples_leaf=clf.best_params_['min_samples_leaf']
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf)
            ,ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

In [ ]:
'''
cross validation RF
'''
classifier="RF"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    

    

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    
    RF = RandomForestClassifier(min_samples_split=300,min_samples_leaf=20,max_depth=8,max_features='sqrt')
    parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
    clf = GridSearchCV(RF, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    n_estimators=clf.best_params_['n_estimators']
    max_depth=clf.best_params_['max_depth']
    min_samples_split=clf.best_params_['min_samples_split']
    min_samples_leaf=clf.best_params_['min_samples_leaf']
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf)
            ,ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

In [ ]:
'''
cross validation NB
'''
classifier="NB"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    

    

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    NB=GaussianNB()
    NB.fit(X_train,Y_train)
    y_pred=NB.predict(X_test)
    y_pred_prob=NB.predict_proba(X_test)
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier,ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

In [ ]:
'''
cross validation LR
'''
classifier="LR"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    

    

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    
    logisReg = LogisticRegression()
    C_range = 10.0 ** np.arange(-4,3,1)
    grid_parame = dict(C=C_range)
    clf = GridSearchCV(logisReg, grid_parame, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    C=clf.best_params_['C']
    y_pred=clf.predict(X_test)
    y_pred_prob=clf.predict_proba(X_test)
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier+"C:"+str(C),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

In [ ]:
'''
cross validation KNN
'''
classifier="KNN"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    

    

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    knn = KNeighborsClassifier()
    #设置k的范围
    k_range = list(range(1,10))
    leaf_range = list(range(1,2))
    weight_options = ['uniform','distance']
    algorithm_options = ['auto','ball_tree','kd_tree','brute']
    param_gridknn = dict(n_neighbors = k_range,weights = weight_options,algorithm=algorithm_options,leaf_size=leaf_range)
    clf = GridSearchCV(knn, param_gridknn, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    n_neighbors=clf.best_params_['n_neighbors']
    weights=clf.best_params_['weights']
    algorithm=clf.best_params_['algorithm']
    leaf_size=clf.best_params_['leaf_size']
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier+"n_neighbors:"+str(n_neighbors)+"weights:"+str(weights)+"algorithm:"+str(algorithm)+"leaf_size:"+str(leaf_size),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')

In [ ]:
'''
cross validation XGBoost
'''
classifier="XGBoost"
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence=path+inputname
RNA_code='ACGU'
# k=4
interval=2
gap=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=division_num,shuffle=False)  

for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
# for interval in xrange(1,k+1):
    final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_x_train))]
    code_values=make_kmer_list(interval,RNA_code)
    code_len=len(code_values)
    positive_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    negative_seq_value=[[0 for jj in xrange(len(seq[0])-interval-gap*(interval-1))] for ii in xrange(code_len)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0] 
                        
                    if c_value==temp_value:
                        positive_seq_value[p][j]+=1
    positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1):
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                        negative_seq_value[p][j]+=1
    negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)/2)
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
    y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-interval-1-gap*(interval-1) :
                for p,c_value in enumerate(code_values):
                    temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                    temp_value=[reduce(lambda x,y:x+y,temp_value)]
                    temp_value=temp_value[0]
                    if c_value==temp_value:
                          y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
                            
    

    

    

#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
    X_train = np.array(final_seq_value)
    Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(y_final_seq_value)
    Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    parameters = [{'n_estimators':range(1,10,1),
                      'max_depth':range(2,10),
                      'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
                      'subsample':[0.75,0.8,0.85,0.9]
                      }]
    clf = GridSearchCV(XGBClassifier(), parameters, cv=10, n_jobs=1, scoring='accuracy')
    clf.fit(X_train, Y_train)
    n_estimators=clf.best_params_['n_estimators']
    max_depth=clf.best_params_['max_depth']
    learning_rate=clf.best_params_['learning_rate']
    subsample=clf.best_params_['subsample']
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"learning_rate:"+str(learning_rate)+"subsample:"+str(subsample),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save(classifier+"_independent_test",[str(X_train.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')